In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
import os
import sys
import json
import argparse
import numpy as np
import math
import time
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import scipy.io
import scipy.fft as fft
import nirsimple as ns
import mne
from scipy.signal import butter, filtfilt
from sklearn.metrics import accuracy_score

from src import sklearn_models, torch_models, featurize
import torch

## Load events from one file

In [80]:
from sklearn.model_selection import train_test_split

In [107]:
events = []
labels = []
# for run in ['2','3','5','7']:
#     events_run = np.load(f"events/events_run{run}.npy")
#     labels_run = np.load(f"events/event_labels_run{run}.npy")
#     events += [i.flatten() for i in events_run]
#     labels += list(labels_run)

for run in ['4','6']:
    events_run = np.load(f"events/events_run{run}-forhead.npy")
    labels_run = np.load(f"events/event_labels_run{run}-forhead.npy")
    print(set(labels_run))
    events += [i.flatten() for i in events_run]
    labels += list(labels_run)
    
counts,bins = np.histogram(labels,5)
print(counts[0]/sum(counts))

{0, 4}
{1, 3, 4}
0.3923076923076923


In [108]:
# X_train, X_test, y_train, y_test = [],[],[],[]
# for run in ['2','3']:
#     events_run = np.load(f"events/events_run{run}.npy")
#     labels_run = np.load(f"events/event_labels_run{run}.npy")
#     X_train += [i.flatten() for i in events_run]
#     y_train += list(labels_run)
    
# for run in ['5','7']:
#     events_run = np.load(f"events/events_run{run}.npy")
#     labels_run = np.load(f"events/event_labels_run{run}.npy")
#     X_test += [i.flatten() for i in events_run]
#     y_test += list(labels_run)

In [109]:
X_train, X_test, y_train, y_test = train_test_split(events, labels, test_size=0.3, random_state=42)


In [110]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [111]:
X_train[0].shape

(48,)

In [112]:
# Predict using the test set
y_pred = clf.predict(X_test)

# Evaluate the classifier's performance
from sklearn.metrics import accuracy_score, classification_report

print(f"Accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%")
print(classification_report(y_test, y_pred))

Accuracy: 40.17%
              precision    recall  f1-score   support

           0       0.66      0.78      0.71        49
           1       0.00      0.00      0.00        19
           3       0.33      0.38      0.36        21
           4       0.04      0.04      0.04        28

    accuracy                           0.40       117
   macro avg       0.26      0.30      0.28       117
weighted avg       0.34      0.40      0.37       117



In [95]:
import matplotlib.pyplot as plt

In [96]:
for run in ['2','3','5','7']:
    labels_run = np.load(f"events/event_labels_run{run}.npy")
    print(set(labels_run))
    print(labels_run)

{0, 1, 2, 4}
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
{0, 2, 3}
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3]
{1, 3, 4}
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 

In [ ]:
counts,bins = np.histogram(np.concatenate([labels2,labels3,labels5,labels7]),bins=5)

In [ ]:
counts,bins

In [ ]:
counts[2] /sum(counts)

In [ ]:
plt.hist(np.concatenate([labels2,labels3,labels5,labels7]),bins=5)

In [ ]:
plt.hist([0,1,2,5,0,2,3,1,1,5,3,5,2,2,2],bins=5)

In [ ]:
events[0].shape

#### Combine epochs into one multi-dimensional array and create labels array

In [ ]:
print("event data epochs, channels, timesteps", len(event_data), len(event_data[0]), len(event_data[0][0]))
print("event lengths", [event.shape[1] for event in event_data])

event_data = np.stack(event_data)
print("stacked shape", event_data.shape)

#Alternating binary labels for each consecutive event (0, 1, 0, 1, ...).
event_labels = np.resize([0,1], len(event_data))

## (optional) Split events into smaller slices to train and test a model to classify

In [ ]:
print("splitting")
#Split the event data into 5 second intervals (since signal is 10hz, that's 10*5=50 time steps)
split_event_data = []
split_event_labels = []
n = 50
for event, label in zip(event_data, event_labels):
    num_parts = len(event[0]) // n
    split_event_data.extend(np.split(event[:, :num_parts * n], num_parts, axis=1))
    split_event_labels.extend([label] * num_parts)
split_event_data = np.stack(split_event_data)
split_event_labels = np.stack(split_event_labels)
print("event data", split_event_data.shape)
print("event lengths", [len(dat) for dat in split_event_data])
print("event labels", len(split_event_labels), split_event_labels)
event_data = split_event_data
print("event data shape", event_data.shape)
event_labels = split_event_labels

## (optional) Generate synthetic data to use to sanity check model performance

In [ ]:
n_synthetic_events = len(event_data)
group_a_frac = .5
group_a_n = round(n_synthetic_events * group_a_frac)
group_b_n = n_synthetic_events - group_a_n
synthetic_event_labels = np.array([0]*group_a_n + [1]*group_b_n)
group_b_idx = group_a_n #last idx of group a is n-1, first idx of group b is n

print("len synthetic event labels:", len(synthetic_event_labels))

synthetic_events = []
synthetic_event_dims = (1, 500)
for event_label in synthetic_event_labels:
    if event_label == 0:
        synthetic_event = np.random.uniform(0,5, synthetic_event_dims)
    else: 
        synthetic_event = np.random.uniform(10,15, synthetic_event_dims)
    synthetic_events.append(synthetic_event)

synthetic_event_data = np.stack(synthetic_events)

noise_scale = 200000
noise = (np.random.random(synthetic_event_data.shape) - np.array(.5)) * noise_scale
synthetic_event_data += noise
print("Shape of staked data:", synthetic_event_data.shape)
print("min,max of class 0", np.min(synthetic_event_data[:][:][:group_b_idx], axis=0), np.max(synthetic_event_data[:][:][:group_b_idx], axis=0))
print("min,max of class 1", np.min(synthetic_event_data[:][:][group_b_idx:], axis=0), np.max(synthetic_event_data[:][:][group_b_idx:], axis=0))

plt.hist(synthetic_event_data[synthetic_event_labels == 0].flatten(), bins=30, alpha=0.5, label='class 0', color='blue')
plt.hist(synthetic_event_data[synthetic_event_labels == 1].flatten(), bins=30, alpha=0.5, label='class 1', color='red')

In [ ]:
print("noise shape", synthetic_event_data.shape)
stds = np.std(synthetic_event_data, axis=1)
print("std shape", stds.shape)
print("stacked shape", np.stack([stds, stds], axis=2).shape)

In [ ]:
event_data = synthetic_event_data
event_labels = synthetic_event_labels

## Generate features

In [ ]:
print("Class proportions", np.bincount(event_labels)/len(event_labels))

In [ ]:
event_data.shape

In [ ]:
data_features = featurize.summary_features(event_data, ["mean", "std"], "channels")
print("Shape of features", data_features.shape)
#print(data_features)

## Train and evaluate models on features

In [ ]:
models, test_accs, train_accs, hps, stats = sklearn_models.machine_learn(
    nirs=data_features,
    labels=event_labels,
    groups=None,
    model="ann",
    normalize=False,
    random_state=None,
    output_folder="outputs",
    n_inner_splits=1,
    n_outer_splits=2
)
print(accs)
print(stats)

In [ ]:
data_features

In [ ]:
idxs = {"test": [1, 3, 4],
        "train": [0, 2]}

for i, model in enumerate(models):
    print("model:", i)
    for type_, idx_list in idxs.items():
        print("type:", type_, idx_list)
        print("labels", event_labels[idx_list])
        print("predictions", model.predict(data_features[idx_list]))
        print("accuracy", accuracy_score(event_labels[idx_list], model.predict(data_features[idx_list])))

In [ ]:
event_data.shape

## Torch models

In [ ]:
model = torch_models.MLP()
optimizer = torch.optim.Adam(model.parameters(), lr=0.000001)
loss_func = torch.nn.CrossEntropyLoss()
trainer = torch_models.Trainer(model, optimizer, loss_func)

In [ ]:
event_labels

In [ ]:
torch.tensor(event_labels, dtype=torch.float64)

In [ ]:
flattened_inputs = event_data.reshape(5, -1)
print(flattened_inputs.shape)
train_dataloader = [(torch.tensor(flattened_inputs[:4], dtype=torch.float32), torch.tensor(event_labels[:4]))]
test_dataloader = [(torch.tensor(flattened_inputs[4:], dtype=torch.float32), torch.tensor(event_labels[4:]))]

In [ ]:
test_dataloader[0][0].shape

In [ ]:
trainer.train(train_dataloader, test_dataloader, 500, 1, 1)

In [ ]:
model(train_dataloader[0][0])